<a href="https://colab.research.google.com/github/UsmanShafeeq/Advanced-Machine-Learning-Project/blob/main/Gender_and_Age_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Enable GPU


In [2]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 802.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00


In [3]:
import tensorflow as tf

# Check GPU availability
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU not found, using CPU.')
else:
    print(f'GPU device found: {device_name}')


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


GPU not found, using CPU.


Step 2: Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Step 3: Install Required Libraries

In [5]:
!pip install tensorflow keras matplotlib seaborn scikit-learn opencv-python tqdm shap lime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.4 MB/s eta 0:00:00
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=9917a1b0d069f1a95046725455f8b1be96f52a8540ac71039639fd2a03f636e8
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


Step 4: Import Libraries

In [6]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, mean_squared_error, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.optimizers import Adam


Step 5: Load and Preprocess Dataset

In [ ]:
# Dataset path (UTKFace or IMDB-WIKI)
dataset_path = '/content/drive/MyDrive/UTKFace/'  # change accordingly

images, ages, genders = [], [], []

for file in tqdm(os.listdir(dataset_path)):
    try:
        age, gender, _ = file.split('_')[:3]
        img = cv2.imread(os.path.join(dataset_path, file))
        img = cv2.resize(img, (128,128))
        images.append(img)
        ages.append(int(age))
        genders.append(int(gender))
    except:
        continue

# Normalize images
images = np.array(images, dtype='float32')/255.0
ages = np.array(ages, dtype='float32')
genders = np.array(genders, dtype='int')

# Train-test split
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(
    images, ages, genders, test_size=0.2, random_state=42
)


  1%|          | 240/23716 [02:31<117:51:02, 18.07s/it]

Step 6: Build Multi-Task CNN Model

In [ ]:
def build_multitask_model(base_model_name='EfficientNetB0', input_shape=(128,128,3)):
    inputs = Input(shape=input_shape)

    if base_model_name == 'VGG16':
        base_model = VGG16(include_top=False, input_tensor=inputs, weights='imagenet')
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(include_top=False, input_tensor=inputs, weights='imagenet')
    else:
        base_model = EfficientNetB0(include_top=False, input_tensor=inputs, weights='imagenet')

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Gender classification output
    gender_output = Dense(1, activation='sigmoid', name='gender_output')(x)
    # Age regression output
    age_output = Dense(1, activation='linear', name='age_output')(x)

    model = Model(inputs=inputs, outputs=[gender_output, age_output])
    model.compile(
        optimizer=Adam(1e-4),
        loss={'gender_output':'binary_crossentropy','age_output':'mse'},
        metrics={'gender_output':'accuracy','age_output':'mae'}
    )
    return model


Step 7: Train Multiple Models

In [ ]:
models_to_train = ['VGG16','ResNet50','EfficientNetB0']
results_list = []

for model_name in models_to_train:
    print(f'\nTraining {model_name}...')
    model = build_multitask_model(model_name)
    history = model.fit(
        X_train, {'gender_output': y_gender_train, 'age_output': y_age_train},
        validation_data=(X_test, {'gender_output': y_gender_test, 'age_output': y_age_test}),
        epochs=15, batch_size=32, verbose=1
    )

    # Predictions
    pred_gender, pred_age = model.predict(X_test)
    pred_gender_labels = (pred_gender > 0.5).astype(int)

    # Metrics
    gender_acc = accuracy_score(y_gender_test, pred_gender_labels)*100
    gender_f1 = f1_score(y_gender_test, pred_gender_labels)
    age_mae = mean_absolute_error(y_age_test, pred_age)
    age_rmse = np.sqrt(mean_squared_error(y_age_test, pred_age))

    results_list.append([model_name, gender_acc, gender_f1, age_mae, age_rmse])

    # Plot Training History
    plt.figure(figsize=(14,5))
    # Gender Accuracy
    plt.subplot(1,2,1)
    plt.plot(history.history['gender_output_accuracy'], label='Train Acc')
    plt.plot(history.history['val_gender_output_accuracy'], label='Val Acc')
    plt.title(f'{model_name} Gender Accuracy')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()

    # Age MAE
    plt.subplot(1,2,2)
    plt.plot(history.history['age_output_mae'], label='Train MAE')
    plt.plot(history.history['val_age_output_mae'], label='Val MAE')
    plt.title(f'{model_name} Age MAE')
    plt.xlabel('Epoch'); plt.ylabel('MAE'); plt.legend()
    plt.show()


Step 8: Result Table

In [ ]:
results_df = pd.DataFrame(results_list, columns=['Model','Gender Accuracy (%)','Gender F1-Score','Age MAE','Age RMSE'])
results_df


Step 9: Visualize Predictions

In [ ]:
pred_gender, pred_age = model.predict(X_test[:12])
pred_gender_labels = (pred_gender>0.5).astype(int)

plt.figure(figsize=(15,8))
for i in range(12):
    plt.subplot(3,4,i+1)
    plt.imshow(X_test[i])
    plt.title(f'True Age: {y_age_test[i]}, Pred Age: {int(pred_age[i])}\nTrue Gender: {y_gender_test[i]}, Pred: {pred_gender_labels[i][0]}')
    plt.axis('off')
plt.tight_layout()
plt.show()


Step 10: Compare Models (Graph)

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.barplot(x='Model', y='Gender Accuracy (%)', data=results_df)
plt.title('Gender Accuracy Comparison'); plt.ylim(0,100)

plt.subplot(1,2,2)
sns.barplot(x='Model', y='Age MAE', data=results_df)
plt.title('Age MAE Comparison'); plt.ylim(0,10)
plt.tight_layout()
plt.show()


Step 11: Grad-CAM for Explainability

In [ ]:
import tensorflow.keras.backend as K

def get_gradcam_heatmap(model, img_array, layer_name='conv5_block3_out'): # For ResNet50
    grad_model = Model([model.inputs], [model.get_layer(layer_name).output, model.output[0]])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:,0]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = K.mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs.numpy()[0]
    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:,:,i] *= pooled_grads[i]
    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap,0)
    heatmap /= np.max(heatmap)
    return heatmap

# Example Grad-CAM
img = np.expand_dims(X_test[0], axis=0)
heatmap = get_gradcam_heatmap(model, img)
plt.imshow(X_test[0])
plt.imshow(heatmap, cmap='jet', alpha=0.5)
plt.title('Grad-CAM Overlay')
plt.axis('off')
plt.show()


Step 12: SHAP Explainability

In [ ]:
import shap
explainer = shap.GradientExplainer(model, X_train[:100])
shap_values = explainer.shap_values(X_test[:5])
shap.image_plot(shap_values, X_test[:5])
